In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import timm

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
from pyts.image import GramianAngularField, MarkovTransitionField
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

2024-11-17 14:35:03.463328: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 14:35:03.471183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-17 14:35:03.480870: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-17 14:35:03.483629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 14:35:03.490603: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# For shuffle
SEED = 42

In [5]:
# For feature engineering
t_d = 30 # detection time
t_w = 10 # time windows

In [6]:
# For model
EPOCH = 30
BATCHSIZE = 32
VAL = 0.2

In [7]:
BASE = '/media/eab301b/b60ac77c-0185-406e-8549-95e9a295e4ca/thesis/thesis_proj/RanSAP/dataset'

In [8]:
DATAPATH = 'original'

In [9]:
FOLDER = "win7-250gb-ssd"

In [10]:
os.chdir(f'{BASE}/{DATAPATH}')
folders = sorted(os.listdir())
print(folders)

['win7-120gb-hdd', 'win7-120gb-ssd', 'win7-250gb-hdd', 'win7-250gb-ssd']


In [11]:
os.chdir(f'{BASE}/{DATAPATH}/{FOLDER}')
labels = sorted(os.listdir())
print(labels)

['AESCrypt', 'Cerber', 'Cerber-largefiles', 'Cerber-w10dirs', 'Darkside', 'Darkside-largefiles', 'Darkside-w10dirs', 'Excel', 'Firefox', 'GandCrab4', 'GandCrab4-largefiles', 'GandCrab4-w10dirs', 'Ryuk', 'Ryuk-largefiles', 'Ryuk-w10dirs', 'SDelete', 'Sodinokibi', 'Sodinokibi-largefiles', 'Sodinokibi-w10dirs', 'TeslaCrypt', 'TeslaCrypt-largefiles', 'TeslaCrypt-w10dirs', 'WannaCry', 'WannaCry-largefiles', 'WannaCry-w10dirs', 'Zip']


In [12]:
benign = ['AESCrypt', 'Zip', 'SDelete', 'Excel', 'Firefox']
ransomware = ['TeslaCrypt', 'TeslaCrypt-largefiles', 'TeslaCrypt-w10dirs',
              'Cerber', 'Cerber-largefiles', 'Cerber-w10dirs', 
              'Darkside', 'Darkside-largefiles', 'Darkside-w10dirs',
              'WannaCry', 'WannaCry-largefiles', 'WannaCry-w10dirs', 
              'GandCrab4', 'GandCrab4-largefiles', 'GandCrab4-w10dirs',
              'Ryuk', 'Ryuk-largefiles', 'Ryuk-w10dirs',
              'Sodinokibi', 'Sodinokibi-largefiles', 'Sodinokibi-w10dirs']

In [13]:
X_train = []
X_test = []
y1_train = []
y1_test = []
window_size = 10

In [14]:
# df_r = pd.read_csv('/media/eab301b/b60ac77c-0185-406e-8549-95e9a295e4ca/thesis/thesis_proj/RanSAP/dataset/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-23-28/ata_read.csv', header=None)
# df_w = pd.read_csv('/media/eab301b/b60ac77c-0185-406e-8549-95e9a295e4ca/thesis/thesis_proj/RanSAP/dataset/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-23-28/ata_write.csv', header=None)
# timestamp_r = np.unique(df_r[0])
# timestamp_w = np.unique(df_r[0])
# #print(df_r)
# print(df_r.shape)
# print(df_w.shape)
# print(timestamp_r.shape)
# print(timestamp_w.shape)
# # filtered_df_r = df_r[(df_r[0] >= timestamp_r[0]) & (df_r[0] < timestamp_r[0+t_w])]
# # print(filtered_df_r.shape)

In [42]:
tmp = []
df_r = pd.read_csv("/media/eab301b/b60ac77c-0185-406e-8549-95e9a295e4ca/thesis/thesis_proj/RanSAP/dataset/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-23-28/ata_read.csv", header=None)
df_w = pd.read_csv("/media/eab301b/b60ac77c-0185-406e-8549-95e9a295e4ca/thesis/thesis_proj/RanSAP/dataset/original/win7-120gb-hdd/AESCrypt/AESCrypt-20200427_16-23-28/ata_write.csv", header=None)
df_r = np.array(df_r)
df_w = np.array(df_w)
# df_r = np.sum(df_r[0:10,3])
# df_w = np.sum(df_w[0:10,3])
# print(df_w)
# print(df_r)

# print(df_r.shape)
# print(df_w.shape)
# print(df_w[0:10,3])
for i in range(16):
    
    # timestamp_r = np.unique(df_r[0])
    # timestamp_w = np.unique(df_w[0])

    # filtered_df_r = df_r[(df_r[0] >= timestamp_r[i]) & (df_r[0] < timestamp_r[i+t_w])]
    # filtered_df_w = df_w[(df_w[0] >= timestamp_w[i]) & (df_w[0] < timestamp_w[i+t_w])]

    # average write throughput [byte/s]
    T_write = np.sum(df_w[i:i+window_size,3])/(np.sum(df_w[0:10,1])/1000000)
    # average read throughput [byte/s]
    T_read = np.sum(df_r[i:i+window_size,3])/(np.sum(df_r[0:10,1])/1000000)

    # variance of logical block addresses (written)
    V_write_mean = np.mean(df_w[i:i+window_size,2])
    V_write = (1/(window_size-1)) * np.sum((df_w[i:i+window_size,2]-V_write_mean)**2)

    # variance of logical block addresses (read)
    V_read_mean = np.mean(df_r[i:i+window_size:2])
    V_read = (1/(window_size-1)) * np.sum((df_r[i:i+window_size:2]-V_read_mean)**2)

    # average normalized Shannon entropy
    H_write = (1/window_size) * np.sum(df_w[i:i+window_size,4])

    tmp.append([T_write, T_read, V_write, V_read, H_write])
print(tmp)


[[15.509913740906637, 4.037691325277679, 380730692293464.1, 1.029052277799915e+18, 0.4223462945045273], [15.509913740906637, 4.037691325277679, 438784249962905.6, 1.0290524446865569e+18, 0.40757960984970976], [15.509913740906637, 4.037691325277679, 455875669430278.44, 1.0303932897564366e+18, 0.40769407649862127], [15.509913740906637, 4.037691325277679, 449934305109222.44, 1.0303934564996908e+18, 0.40550072749477045], [15.509913740906637, 4.037691325277679, 414622700620106.6, 1.0312256346691035e+18, 0.4051392274105731], [15.509913740906637, 4.037691325277679, 354425254913695.25, 1.0312257161318843e+18, 0.4049290826788261], [15.509913740906637, 4.037691325277679, 269341967989988.22, 1.0307175769226424e+18, 0.404541153337153], [15.509913740906637, 4.037691325277679, 159372839848985.56, 1.030717571157676e+18, 0.37825458874888934], [15.509913740906637, 4.037691325277679, 28347046582283.375, 1.0302091813793495e+18, 0.375958299890819], [15.509913740906637, 4.037691325277679, 28703106387115.37

In [ ]:
window_size = 10
np.random.seed(SEED)
for folder in folders:
    for label in labels:
        os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}')
        dirs = sorted(os.listdir())
        dirs = np.array(dirs)
        # Shuffle directory
        np.random.seed(SEED)
        np.random.shuffle(dirs)
        # train_idx = int(len(dirs)*0.8)
        # print(f"Train index: {train_idx}")

        for dir_idx in range(len(dirs)):
            print(dirs[dir_idx])
            os.chdir(f'{BASE}/{DATAPATH}/{folder}/{label}/{dirs[dir_idx]}')
            files = sorted(os.listdir())
            tmp = []
            
            for i in range():
                df_r = pd.read_csv(f'{BASE}/{DATAPATH}/{label}/{dirs[dir_idx]}/{files[0]}', header=None)
                df_w = pd.read_csv(f'{BASE}/{DATAPATH}/{label}/{dirs[dir_idx]}/{files[1]}', header=None)
                # timestamp_r = np.unique(df_r[0])
                # timestamp_w = np.unique(df_w[0])

                # filtered_df_r = df_r[(df_r[0] >= timestamp_r[i]) & (df_r[0] < timestamp_r[i+t_w])]
                # filtered_df_w = df_w[(df_w[0] >= timestamp_w[i]) & (df_w[0] < timestamp_w[i+t_w])]

                # average write throughput [byte/s]
                T_write = np.sum(df_w[i:i+window_size,3])/window_size

                # average read throughput [byte/s]
                T_read = np.sum(df_r[i:i+window_size:3])/window_size

                # variance of logical block addresses (written)
                V_write_mean = np.mean(df_w[i:i+window_size,2])
                V_write = (1/(window_size-1)) * np.sum((df_w[i:i+window_size,2]-V_write_mean)**2)

                # variance of logical block addresses (read)
                V_read_mean = np.mean(df_r[i:i+window_size:3])
                V_read = (1/(window_size-1)) * np.sum((df_r[i:i+window_size:3]-V_read_mean)**2)

                # average normalized Shannon entropy
                H_write = (1/window_size) * np.sum(df_w[i:i+window_size,4])

                tmp.append([T_write, T_read, V_write, V_read, H_write])

            # # Train-Test split
            # if dir_idx < train_idx:
            #     X_train.append(tmp)
            #     y1_train.append(label)
            # else:
            #     X_test.append(tmp)
            #     y1_test.append(label)

NameError: name 'np' is not defined

In [40]:
X_train = np.array(X_train)
y1_train = np.array(y1_train)
X_test = np.array(X_test)
y1_test = np.array(y1_test)
print(type(X_train))
print(X_train.shape)
print(y1_train.shape)
print(X_test.shape)
print(y1_test.shape)


<class 'numpy.ndarray'>
(0,)
(0,)
(0,)
(0,)


In [179]:
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_reshaped.shape

(13312, 5)

In [180]:
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_reshaped.shape

(3392, 5)

In [181]:
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train_reshaped)
X_test_normalized = scaler.transform(X_test_reshaped)

In [182]:
X_train_normalized = X_train_normalized.reshape(X_train.shape)
X_test_normalized = X_test_normalized.reshape(X_test.shape)

In [183]:
print(X_train_normalized.shape)
print(X_test_normalized.shape)

(832, 16, 5)
(212, 16, 5)


In [184]:
np.save('E:/thesis project/thesis_proj/X_train_normalized.npy',X_train_normalized)
np.save('E:/thesis project/thesis_proj/y1_train.npy',y1_train)
np.save('E:/thesis project/thesis_proj/X_test_normalized.npy',X_test_normalized)
np.save('E:/thesis project/thesis_proj/y1_test.npy',y1_test)

## Label Encoder

In [ ]:
# y_train = []
# for y in y1_train:
#     if y in benign:
#         y_train.append('Benign')
#     else:
#         y_train.append('Ransomware')
# print(len(y_train))

832


In [ ]:
# y_test = []

# for y in y1_test:
#     if y in benign:
#         y_test.append('Benign')
#     else:
#         y_test.append('Ransomware')

In [ ]:
y_train = np.array(y1_train)
y_test = np.array(y1_test)

In [188]:
lbl = LabelEncoder()

In [189]:
classes, count = np.unique(y_train, return_counts=True)
print(classes)
print(count)

['Benign' 'Ransomware']
[160 672]


In [190]:
print(lbl.fit_transform(classes), classes)
y_train_int = lbl.fit_transform(y_train)
y_train_int.shape

[0 1] ['Benign' 'Ransomware']


(832,)

In [191]:
classes_test, count_test = np.unique(y_test, return_counts=True)
print(classes_test)
print(count_test)

['Benign' 'Ransomware']
[ 41 171]


In [192]:
y_test_int = lbl.transform(y_test)
y_test_int.shape

(212,)

## Data Augmentation (SMOTE)

In [193]:
X_train_normalized = np.load('E:/thesis project/thesis_proj/X_train_normalized.npy')
X_test_normalized = np.load('E:/thesis project/thesis_proj/X_test_normalized.npy')
X_train_normalized = np.load('E:/thesis project/thesis_proj/X_train_normalized.npy')
y1_test = np.load('E:/thesis project/thesis_proj/y1_test.npy')
y1_train = np.load('E:/thesis project/thesis_proj/y1_train.npy')
# For model
EPOCH = 30
BATCHSIZE = 32
VAL = 0.2
BASE = 'E:/thesis project/thesis_proj/'

In [194]:
data_train_flattened = X_train_normalized.reshape(X_train_normalized.shape[0], -1)
data_train_flattened.shape

(832, 80)

In [195]:
SEED =42
np.random.seed(SEED)
smote = SMOTE(sampling_strategy='auto', random_state=SEED) # Resample benign class to be equal to ransomware class
data_train_resampled_flattened, y_train_int_aug = smote.fit_resample(data_train_flattened, y_train_int)
print(y_train_int_aug.shape)
np.save('E:/thesis project/thesis_proj/y_train_int_aug.npy',y_train_int_aug)

(1344,)


In [196]:
X_train_normalized_aug = data_train_resampled_flattened.reshape(data_train_resampled_flattened.shape[0], X_train_normalized.shape[1], X_train_normalized.shape[2])
X_train_normalized_aug.shape
np.save('E:/thesis project/thesis_proj/X_train_normalized_aug.npy',X_train_normalized_aug)